In [1]:
import multiprocessing
import sys

import numpy as np
import pytorch_lightning as pl
import sklearn
import torch
import torchvision

from src.data.dataloaders import ImagesDataset
from src.models.model import SelfSupervisedLearner
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA
from torch.utils.data import DataLoader

BATCH_SIZE = 256
EPOCHS     = 1000
LR         = 3e-4
IMAGE_SIZE = 96 # Change this depending on dataset
NUM_GPUS= 0 # Change this depending on host
NUM_WORKERS = multiprocessing.cpu_count()

In [3]:
resnet = torchvision.models.resnet18(pretrained=False)
model = SelfSupervisedLearner(
    resnet,
    image_size = IMAGE_SIZE,
    hidden_layer = 'avgpool',
    projection_size = 256,
    projection_hidden_size = 4096,
    moving_average_decay = 0.99,
    lr = LR
)

    
argv = ["train.py", "--load", "./ckpt/learner_0510_v100.pt"]
model.load_state_dict(torch.load(argv[2]))
print("Loaded checkpoint from ", argv[2])

#TODO: for some reason labels don't exist in my wget data 
#ds = ImagesDataset("./dataset/test_images", IMAGE_SIZE, train=False)
data_transforms = torchvision.transforms.Compose([torchvision.transforms.ToTensor()])
train_dataset = torchvision.datasets.STL10('./dataset/train_split', split='train', download=False,
                   transform=data_transforms)
train_loader = DataLoader(train_dataset, batch_size=5000, num_workers=NUM_WORKERS, shuffle=False)


train_imgs, train_labels = next(iter(train_loader))
print("Train loading done")

test_dataset = torchvision.datasets.STL10('./dataset/test_split', split='test', download=False, transform=data_transforms)
test_loader = DataLoader(test_dataset, batch_size=8000, num_workers=NUM_WORKERS, shuffle=False)
test_imgs, test_labels = next(iter(test_loader))
print("Test loading done")

#train_projs, train_embeddings = model.learner.forward(train_imgs, return_embedding=True)
#test_projs, test_embeddings = model.learner.forward(test_imgs, return_embedding=True)

print("got embeddings")

Loaded checkpoint from  ./ckpt/learner_0510_v100.pt
Train loading done
Test loading done
got embeddings


In [5]:
X = torch.unsqueeze(train_imgs[0], 0)
print(X.shape)

model.eval()

losses_train = []

loss = model.learner.forward(train_imgs, return_losses=True)

"""
for i, img in enumerate(train_imgs[:100]):
    loss = model.learner.forward(torch.unsqueeze(img, 0))
    losses_train.append(loss)
    
    if (i % 100) == 0: print(i)
        
print(losses_train)
"""

torch.Size([1, 3, 96, 96])


'\nfor i, img in enumerate(train_imgs[:100]):\n    loss = model.learner.forward(torch.unsqueeze(img, 0))\n    losses_train.append(loss)\n    \n    if (i % 100) == 0: print(i)\n        \nprint(losses_train)\n'

tensor(0.6558, grad_fn=<SelectBackward>)